In [5]:
%pip install -r ../requirements.txt

ERROR: Invalid requirement: '<<<<<<< HEAD': Expected package name at the start of dependency specifier
    <<<<<<< HEAD
    ^ (from line 10 of ../requirements.txt)
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import re
from sklearn.pipeline import Pipeline # Pipeline applies a list of transforms. You can also add an estimator at the end, so it will be completely encapsulated.
from sklearn.preprocessing import FunctionTransformer # FunctionTransformer allows to apply an arbitrary function to the data, so we can use it in the pipeline
import unicodedata
import spacy
from typing import List

In [7]:
path = "../llm-proper-data/train_drcat_"

In [14]:
drcat_01 = pd.read_csv(path + "01.csv")
drcat_02 = pd.read_csv(path + "02.csv")
drcat_03 = pd.read_csv(path + "03.csv")
drcat_04 = pd.read_csv(path + "04.csv")

In [9]:
drcat_01.head()

,text,label,source,fold
0,There are alot reasons to keep our the despise...,0,persuade_corpus,2
1,Driving smart cars that drive by themself has ...,0,persuade_corpus,4
2,"Dear Principal,\n\nI believe that students at ...",0,persuade_corpus,0
3,"Dear Principal,\n\nCommunity service should no...",0,persuade_corpus,0
4,My argument for the development of the driverl...,0,persuade_corpus,3


In [10]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 44.7 MB/s eta 0:00:00MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [11]:
def remove_excessive_spaces(text: str) -> str:
    """
    This function removes excessive spaces from the text.

    Args:
        text (str): The input text.

    Returns:
        str: The text with excessive spaces removed.
    """
    return re.sub(r'\s+', ' ', text).strip() 

def remove_repeated_non_word_characters(text: str) -> str:
    """
    This function removes repeated non-word characters from the text.

    Args:
        text (str): The input text.

    Returns:
        str: The text with repeated non-word characters removed.
    """
    return re.sub(r'(\W)\1+', r'\1', text).strip()

def remove_first_line_from_text(text: str) -> str:
    """
    This function removes the first line from the text.

    Args:
        text (str): The input text.

    Returns:
        str: The text with the first line removed.
    """
    return re.sub(r'^.*\n', '', text).strip()

def remove_last_line_from_text(text: str) -> str:
    """
    This function removes the last line from the text.

    Args:
        text (str): The input text.

    Returns:
        str: The text with the last line removed.
    """
    return re.sub(r'\n.*$', '', text).strip()

def fix_isolated_commas_in_text(text: str) -> str:
    """
    This function fixes isolated commas in the text.

    Args:
        text (str): The input text.

    Returns:
        str: The text with isolated commas fixed.
    """
    text = re.sub(r' ([.,:;!?])', r'\1', text)
    return text.strip()

def keep_words_longer_than(text: str, min_length: int = 2) -> str:
    """
    This function keeps only the words in the text that are longer than a given length.

    Args:
        text (str): The input text.
        min_length (int, optional): The minimum length of the words to keep. Defaults to 2.

    Returns:
        str: The text with only the words longer than the given length.
    """
    return ' '.join([word for word in text.split() if len(word) > min_length])

def keep_only_alphabet_characters(text: str) -> str:
    """
    This function keeps only the alphabet characters in the text.

    Args:
        text (str): The input text.

    Returns:
        str: The text with only the alphabet characters.
    """
    return re.sub(r'[^a-zA-Z]', ' ', text).strip()

def remove_accents_from_text(text: str) -> str:
    """
    This function removes accents from the text.

    Args:
        text (str): The input text.

    Returns:
        str: The text with accents removed.
    """
    return unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')

def lemmatize_text_with_spacy(text: str) -> str:
    """
    This function lemmatizes the text using the Spacy library.

    Args:
        text (str): The input text.

    Returns:
        str: The lemmatized text.
    """
    doc = nlp_spacy(text)
    return ' '.join([token.lemma_ for token in doc])


pipeline_clean_text = Pipeline([
    ('remove_first_line_from_text', FunctionTransformer(remove_first_line_from_text)),
    ('remove_last_line_from_text', FunctionTransformer(remove_last_line_from_text)),
    ('remove_excessive_spaces', FunctionTransformer(remove_excessive_spaces)),
    ('remove_repeated_non_word_characters', FunctionTransformer(remove_repeated_non_word_characters)),
    ('fix_isolated_commas_in_text', FunctionTransformer(fix_isolated_commas_in_text)),
])

nlp_spacy = spacy.load('en_core_web_sm')

In [18]:
drcat_01['clean_text'] = drcat_01['text'].apply(pipeline_clean_text.transform)
drcat_02['clean_text'] = drcat_02['text'].apply(pipeline_clean_text.transform)
drcat_03['clean_text'] = drcat_03['text'].apply(pipeline_clean_text.transform)
drcat_04['clean_text'] = drcat_04['text'].apply(pipeline_clean_text.transform)

In [26]:
df = pd.concat([drcat_01, drcat_02, drcat_03, drcat_04], axis=0, ignore_index=True)
df = df.drop(columns=['essay_id','source','fold'])
df = df.drop_duplicates()

In [27]:
df.head()

,text,label,clean_text,prompt
0,There are alot reasons to keep our the despise...,0,Another reason is that the Electiral College r...,NaN
1,Driving smart cars that drive by themself has ...,0,First let's start off with the pros of having ...,NaN
2,"Dear Principal,\n\nI believe that students at ...",0,I believe that students at school should not b...,NaN
3,"Dear Principal,\n\nCommunity service should no...",0,Community service should not be required by al...,NaN
4,My argument for the development of the driverl...,0,"First of all, the diverless cars will help ton...",NaN
